# Introduction
[Reference](https://www.hackerrank.com/challenges/fibonacci-finding-easy/problem?isFullScreen=true)

You're given three numbers: $a$, $b$, and $N$, and all you have to do is to find the number $F_N$ where
- $F_0 = A$
- $F_1 = B$
- $F_N = F_{N-1} + F_{N-2}$

#### Constraints
Also, you're given the number of tests $T$ and our algorithm need to satisfies these conditions
- $1 \leq A,B,N \leq 10^9$
- $10 \leq T \leq 1000$

### 1. Basic Fibonacci approach by induction

In [2]:
from math import sqrt

def basic_induction_fibo(a, b, n):
    if n == 0:
        return a
    elif n == 1:
        return b
    else:
        return basic_induction_fibo(a, b, n-1) + basic_induction_fibo(a, b, n-2)
    
print([basic_induction_fibo(3, 7, k) for k in range(8)])

[3, 7, 10, 17, 27, 44, 71, 115]


In [3]:
def isFibo(n):
    phi = (1 + sqrt(5))/2
    phi_m = (1 - sqrt(5))/2
    res = lambda n: (phi**n - phi_m**n) / sqrt(5)

    count = 0
    for k in range(100):
        if int(res(k)) == n:
            count += 1
    if count == 0:
        return "IsNotFibo"
    else:
        return "IsFibo"

check_list = [3, 5, 7, 8, 9, 16]
for num in check_list:
    print(f"{num} : {isFibo(num)}")

3 : IsFibo
5 : IsFibo
7 : IsNotFibo
8 : IsFibo
9 : IsNotFibo
16 : IsNotFibo


##### Disadvantage

What happened if you have a large-numbers??

In [4]:
basic_induction_fibo(123123, 456456, 33)

1877014714575

### 2. Using golden-ratio
When $a = b = 1$, we have
$${\displaystyle F_{n}={\frac {1}{\sqrt {5}}}\left[{\Big (}{\frac {1+{\sqrt {5}}}{2}}{\Big )}^{n}-{\Big (}{\frac {1-{\sqrt {5}}}{2}}{\Big )}^{n}\right]}$$

Also, we observed that

| n th | Standard Fibonacci | General Fibo sequence |
|:-:|:-:|:-:|
|0 |1 | `a` |
|1 |1 | `b` |
|2 |2 | `a + b`|
|3 |3 | `a + 2b` |
|4 |5 | `2a + 3b` |
|5 |8 | `3a + 5b` |
|6 |13 | `5a + 8b` |
|7 |21 | `8a + 13b` |
|8 |34 | `13a + 21b` |
|9 |55 | `21a + 34b` |
|..|..|..|
|n|F(n)| `a*F(n-2) + b*F(n-1)`|

In [5]:
def fibo_by_golden_rt(a, b , n):

    def fibo_coef(n):
        n = n+1
        return int((phi**n - (1 - phi)**n) / sqrt(5))
    phi = (1 + sqrt(5)) / 2
    fa = fibo_coef(n-2)
    fb = fibo_coef(n-1)
    if n == 0:
        return a
    else:
        return (a*fa + b*fb)

print([basic_induction_fibo(3, 7, k) for k in range(8)])
print([fibo_by_golden_rt(3, 7, _) for _ in range(8)])

print("And for a=123123, b=456456, n=789, we have Fibo(n) is\n", fibo_by_golden_rt(123123, 456456, 789))

[3, 7, 10, 17, 27, 44, 71, 115]
[3, 7, 10, 17, 27, 44, 71, 115]
And for a=123123, b=456456, n=789, we have Fibo(n) is
 185405966609566795431380389565318287946392238591171264295594384033467591688082277158577597818946037814941223025710495618048446347788346396669611353571240593032529454301184


#### Disadvantage of this method?
Also, what happened if the number is large enough? 

In [6]:
fibo_by_golden_rt(509618737, 460201239, 229176339)

OverflowError: (34, 'Numerical result out of range')

The `OverflowError` is caused by the float number $\phi$ that defined while computing $F(n-2)$ can not represented in Python for $N$ large enough

### 3. Using matrix-form + modulo for large number

$$ {\displaystyle {F_{k+2} \choose F_{k+1}}={\begin{pmatrix}1&1\\1&0\end{pmatrix}}{F_{k+1} \choose F_{k}}} $$

In [ ]:
mod = int(1e9 + 7)

def mul(A, B):
    """ Matrix multiplication between 2 matrix A and B """
    res = [[0, 0], [0, 0]]
    for i in range(2):
        for j in range(2):
            for h in range(2):
                res[i][j] = res[i][j] + A[i][h] * B[h][j]
                res[i][j] %= mod
    return res

def fibo(a, b, n):
    res = [[1, 0], [0, 1]]
    x = [[0, 1], [1, 1]]

    while n:
        if (n % 2) == 1:
            res = mul(res, x)
        x = mul(x, x)
        n //= 2
    return (res[0][0] * a + res[0][1] * b) % mod

a, b, n = 509618737, 460201239, 229176339    
print( fibo(a, b, n) )

945141656


In [ ]:
check_lists = [ 
                (2, 3, 1),
                (9, 1, 7),
                (9, 8, 3),
                (2, 4, 9),
                (1, 7, 2),
                (1, 8, 1),
                (4, 3, 1),
                (3, 7, 5)
                ]

for test_case in check_lists:
    a, b, n = test_case
    print(f"For F0: {a} \t F1: {b} \t\t then F({n}) := {fibo(a, b, n)}")

For F0: 2 	 F1: 3 		 then F(1) := 3
For F0: 9 	 F1: 1 		 then F(7) := 85
For F0: 9 	 F1: 8 		 then F(3) := 25
For F0: 2 	 F1: 4 		 then F(9) := 178
For F0: 1 	 F1: 7 		 then F(2) := 8
For F0: 1 	 F1: 8 		 then F(1) := 8
For F0: 4 	 F1: 3 		 then F(1) := 3
For F0: 3 	 F1: 7 		 then F(5) := 44


In [ ]:
check_lists = [ 
                (123123, 456456, 789),
                (65880447, 13069821, 793896530),
                (920881302, 970435252, 891913774),
                (811504813, 578511105, 879804865),
                (52239400, 997333914, 27011537),
                (362554139, 179703539, 700690282),
                (798212938, 664795656, 47975595),
                (905159409, 466169640, 973854593),
                (388863176, 397784067, 515732595),
                (822879041, 296015172, 403426974),
                (333749099, 332748708, 230988316),
                (301528301, 970005191, 932369784),
                (466204053, 964295783, 75594258),
                (826692474, 79629127, 463898065),
                (6112363, 202545231, 15391640),
                (226237249, 163941961, 759159039),
                (178448624, 275620289, 972495101)
                ]

for test_case in check_lists:
    a, b, n = test_case
    print(f"For Fibo(0): {a} \t Fibo(1): {b} \t\t then Fibo({n}) := {fibo(a, b, n)}")

For Fibo(0): 123123 	 Fibo(1): 456456 		 then Fibo(789) := 893340029
For Fibo(0): 65880447 	 Fibo(1): 13069821 		 then Fibo(793896530) := 43375536
For Fibo(0): 920881302 	 Fibo(1): 970435252 		 then Fibo(891913774) := 49437980
For Fibo(0): 811504813 	 Fibo(1): 578511105 		 then Fibo(879804865) := 914989711
For Fibo(0): 52239400 	 Fibo(1): 997333914 		 then Fibo(27011537) := 551574016
For Fibo(0): 362554139 	 Fibo(1): 179703539 		 then Fibo(700690282) := 703639074
For Fibo(0): 798212938 	 Fibo(1): 664795656 		 then Fibo(47975595) := 953582746
For Fibo(0): 905159409 	 Fibo(1): 466169640 		 then Fibo(973854593) := 421520897
For Fibo(0): 388863176 	 Fibo(1): 397784067 		 then Fibo(515732595) := 279855686
For Fibo(0): 822879041 	 Fibo(1): 296015172 		 then Fibo(403426974) := 26213385
For Fibo(0): 333749099 	 Fibo(1): 332748708 		 then Fibo(230988316) := 257166781
For Fibo(0): 301528301 	 Fibo(1): 970005191 		 then Fibo(932369784) := 849051703
For Fibo(0): 466204053 	 Fibo(1): 964295783 		 t